## Instalación de dependencias

In [ ]:
# Ejecutar solo si es la primera vez
%pip install pandas
%pip install mlflow
%pip install openai
%pip install python-dotenv
%pip install gradio
%pip install scikit-learn

## Importar librerias

In [ ]:
import pandas as pd
import numpy as np
import mlflow

import sys
sys.path.append('../project')  # ruta relativa o absoluta
import utils


from sklearn.model_selection import train_test_split

## Cargar el dataset

In [98]:
df = pd.read_csv("../data/winequality-white.csv", sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [91]:
df.describe()

,"fixed acidity;""volatile acidity"";""citric acid"";""residual sugar"";""chlorides"";""free sulfur dioxide"";""total sulfur dioxide"";""density"";""pH"";""sulphates"";""alcohol"";""quality"""
count,4898
unique,3961
top,7.3;0.19;0.27;13.9;0.057;45;155;0.99807;2.94;0...
freq,8


Observamos que cada columna del archivo .csv, se está procesando como todo parte de una misma columna, a su vez, vemos que los datos de cada columna están separados por un ´;´ así que vamos a separar los datos correctamente en las columnas correspondientes, para luego ver si hace falta una limpieza en los datos

In [92]:
def save_name_columns(df: pd.DataFrame) -> list:
    return df.columns.tolist()[0].replace(';"', ';').replace('";', ';').strip('"').split(';')

In [93]:
def separate_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Separar las columnas usando ';' como delimitador
    df_separated = df.iloc[:, 0].str.split(';', expand=True)
    # Asignar nombres de columnas adecuados
    df_separated.columns = save_name_columns(df)
    return df_separated

Ahora que separamos el dataset en diferentes columnas con sus respectivos valores, vamos a ver algo de información, porque lo más seguro es que las columnas sean de tipo "object" cuando deberían ser de tipo "float".

In [94]:
df = utils.separate_columns(df)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   fixed acidity         4898 non-null   object
 1   volatile acidity      4898 non-null   object
 2   citric acid           4898 non-null   object
 3   residual sugar        4898 non-null   object
 4   chlorides             4898 non-null   object
 5   free sulfur dioxide   4898 non-null   object
 6   total sulfur dioxide  4898 non-null   object
 7   density               4898 non-null   object
 8   pH                    4898 non-null   object
 9   sulphates             4898 non-null   object
 10  alcohol               4898 non-null   object
 11  quality               4898 non-null   object
dtypes: object(12)
memory usage: 459.3+ KB


Efectivamente observamos que no son del tipo indicado, vamos a corregirlo

In [95]:
def change_dtype(df: pd.DataFrame) -> pd.DataFrame:
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

In [96]:
df = utils.change_dtype(df)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Observamos también que no hay elementos nulos porque podemos observar que hay 4898 elementos, y cada columna tiene 4898 elementos no nulos. Entonces, con este resultado vamos a separar los datos para entrenamiento y para prueba.